# Install

In [2]:
# pip install labelme

In [4]:
# pip install opencv-python-headless

In [6]:
# pip install scikit-image

# Jason to metrix

Function

In [17]:
import math
import uuid

import numpy as np
import PIL.Image
import PIL.ImageDraw

from labelme.logger import logger

In [18]:
def polygons_to_mask(img_shape, polygons, shape_type=None):
    logger.warning(
        "The 'polygons_to_mask' function is deprecated, "
        "use 'shape_to_mask' instead."
    )
    return shape_to_mask(img_shape, points=polygons, shape_type=shape_type)


def shape_to_mask(
    img_shape, points, shape_type=None, line_width=10, point_size=5
):
    mask = np.zeros(img_shape[:2], dtype=np.uint8)
    mask = PIL.Image.fromarray(mask)
    draw = PIL.ImageDraw.Draw(mask)
    xy = [tuple(point) for point in points]
    if shape_type == "circle":
        assert len(xy) == 2, "Shape of shape_type=circle must have 2 points"
        (cx, cy), (px, py) = xy
        d = math.sqrt((cx - px) ** 2 + (cy - py) ** 2)
        draw.ellipse([cx - d, cy - d, cx + d, cy + d], outline=1, fill=1)
    elif shape_type == "rectangle":
        assert len(xy) == 2, "Shape of shape_type=rectangle must have 2 points"
        draw.rectangle(xy, outline=1, fill=1)
    elif shape_type == "line":
        assert len(xy) == 2, "Shape of shape_type=line must have 2 points"
        draw.line(xy=xy, fill=1, width=line_width)
    elif shape_type == "linestrip":
        draw.line(xy=xy, fill=1, width=line_width)
    elif shape_type == "point":
        assert len(xy) == 1, "Shape of shape_type=point must have 1 points"
        cx, cy = xy[0]
        r = point_size
        draw.ellipse([cx - r, cy - r, cx + r, cy + r], outline=1, fill=1)
    else:
        assert len(xy) > 2, "Polygon must have points more than 2"
        draw.polygon(xy=xy, outline=1, fill=1)
    mask = np.array(mask, dtype=bool)
    return mask


def shapes_to_label(img_shape, shapes, label_name_to_value):
    cls = np.zeros(img_shape[:2], dtype=np.int32)
    ins = np.zeros_like(cls)
    instances = []
    for shape in shapes:
        points = shape["points"]
        label = shape["label"]
        group_id = shape.get("group_id")
        if group_id is None:
            group_id = uuid.uuid1()
        shape_type = shape.get("shape_type", None)

        cls_name = label
        instance = (cls_name, group_id)

        if instance not in instances:
            instances.append(instance)
        ins_id = instances.index(instance) + 1
        cls_id = label_name_to_value[cls_name]

        mask = shape_to_mask(img_shape[:2], points, shape_type)
        cls[mask] = cls_id
        ins[mask] = ins_id

    return cls, ins


def labelme_shapes_to_label(img_shape, shapes):
    logger.warn(
        "labelme_shapes_to_label is deprecated, so please use "
        "shapes_to_label."
    )

    label_name_to_value = {"_background_": 0}
    for shape in shapes:
        label_name = shape["label"]
        if label_name in label_name_to_value:
            label_value = label_name_to_value[label_name]
        else:
            label_value = len(label_name_to_value)
            label_name_to_value[label_name] = label_value

    lbl, _ = shapes_to_label(img_shape, shapes, label_name_to_value)
    return lbl, label_name_to_value


def masks_to_bboxes(masks):
    if masks.ndim != 3:
        raise ValueError(
            "masks.ndim must be 3, but it is {}".format(masks.ndim)
        )
    if masks.dtype != bool:
        raise ValueError(
            "masks.dtype must be bool type, but it is {}".format(masks.dtype)
        )
    bboxes = []
    for mask in masks:
        where = np.argwhere(mask)
        (y1, x1), (y2, x2) = where.min(0), where.max(0) + 1
        bboxes.append((y1, x1, y2, x2))
    bboxes = np.asarray(bboxes, dtype=np.float32)
    return bboxes

version 5

In [19]:
import json
import matplotlib.pyplot as plt
import numpy as np
from skimage.transform import resize
import cv2

In [20]:
#draw json function
def json_to_matrix(path,size):
    with open(path, "r",encoding="utf-8") as f:
        dj = json.load(f)

    #Generate metrix [0]*size(img_height, img_width) 
    lung_lobe = np.zeros((dj['imageHeight'], dj['imageWidth']))
    image_Lesion = np.zeros((dj['imageHeight'], dj['imageWidth']))
    '''
    Label class
    RUL :right upper lobe:   1
    RLL :right lower lobe:   2
    RML :right middle lobe:  3
    LUL :left upper lobe:    4
    LLL :left lower lobe:    5
    Lesion :left lower lobe: 6
    '''

  #Draw point in metrix above
    for i in range(len(dj['shapes'])):
        mask = shape_to_mask((dj['imageHeight'],dj['imageWidth']), dj['shapes'][i]['points'], shape_type=None,line_width=1, point_size=1)
        mask_img = mask.astype(np.int)#boolean to 0,Convert to 1
        if dj['shapes'][i]['label'] == "RUL": 
            lung_lobe = lung_lobe + mask_img*1
        elif dj['shapes'][i]['label'] == "RLL":
            lung_lobe = lung_lobe + mask_img*2
        
        lung_lobe = np.where(lung_lobe==3, 2, lung_lobe) # pixel อนทับกันละหว่าง RUL, RLL =3เลยให้ label เป็น RLL (2) แทน
            
    for i in range(len(dj['shapes'])):
        mask = shape_to_mask((dj['imageHeight'],dj['imageWidth']), dj['shapes'][i]['points'], shape_type=None,line_width=1, point_size=1)
        mask_img = mask.astype(np.int)#boolean to 0,Convert to 1
        if dj['shapes'][i]['label'] == "RML":
            lung_lobe = lung_lobe + mask_img*3
        
        lung_lobe = np.where(lung_lobe==5, 3, lung_lobe) # pixel อนทับกันละหว่าง RLL, RML =5เลยให้ label เป็น RML (3) แทน
        lung_lobe = np.where(lung_lobe==6, 3, lung_lobe) # pixel อนทับกันละหว่าง RUL, RLL, RML=6เลยให้ label เป็น RML (3) แทน
        
  
    for i in range(len(dj['shapes'])):
        mask = shape_to_mask((dj['imageHeight'],dj['imageWidth']), dj['shapes'][i]['points'], shape_type=None,line_width=1, point_size=1)
        mask_img = mask.astype(np.int)#boolean to 0,Convert to 1
        
        if dj['shapes'][i]['label'] == "LUL":
            lung_lobe = lung_lobe + mask_img*4
        elif dj['shapes'][i]['label'] == "LLL":
            lung_lobe = lung_lobe + mask_img*5
        elif dj['shapes'][i]['label'] == "Lesion":
            image_Lesion = image_Lesion + mask_img
        
        lung_lobe = np.where(lung_lobe==8, 4, lung_lobe) # pixel ซ้อนทับกันละหว่าง LUL, LUL =8เลยให้ label เป็น LUL (4) แทน
        lung_lobe = np.where(lung_lobe==9, 5, lung_lobe) # pixel ซ้อนทับกันละหว่าง LUL, LLL =9เลยให้ label เป็น LLL (5) แทน
        lung_lobe = np.where(lung_lobe==10, 5, lung_lobe) # pixel ซ้อนทับกันละหว่าง LLL, LLL =10 เลยให้ label เป็น LLL (5) แทน
        image_Lesion =  np.where(image_Lesion>=2, 1, image_Lesion) #บางทีlabelทับกัน

    # resize to 128x128
    lung_lobe = cv2.resize(lung_lobe, (size, size), interpolation = cv2.INTER_NEAREST) # cv2.INTER_AREA for resize image
    image_Lesion = cv2.resize(image_Lesion, (size, size), interpolation = cv2.INTER_NEAREST)
  
    # combine Lung Lobe and Lesion
    lung_lobe_Lesion = image_Lesion.copy()
    for x in range(len(image_Lesion)):
        for y in range(len(image_Lesion)):
            if image_Lesion[y][x] == 0:
                lung_lobe_Lesion[y][x] = lung_lobe[y][x]
            else:
                lung_lobe_Lesion[y][x] = 6
    
    
    return (np.asarray(lung_lobe, dtype="uint8"), np.asarray(image_Lesion, dtype="uint8"), np.asarray(lung_lobe_Lesion, dtype="uint8"))

# Make Mask (numpy array) for loop

In [21]:
# check path
import os
os.getcwd()

'/root/Project/LungLobe'

### Path setting

In [36]:
path_ = '/tf/Project/LungLobe/'
# dataset = 'Verification Lot 1'
dataset = 'Verification Lot 2'
# dataset = 'Unverified'
path = path_+dataset
path

'/tf/Project/LungLobe/Verification Lot 2'

In [37]:
dirPath = [i for i in os.listdir(path)]
dirPath.remove('.ipynb_checkpoints')
print(len(dirPath))
print(dirPath)

9
['Mild 08', 'Moderate 03', 'Moderate 06', 'Moderate 07', 'Moderate 08', 'Severe 02', 'Severe 03', 'Severe 07', 'Severe 08']


### Save path

In [38]:
# save_folder = 'Tif_256x256/'
# save_folder = 'Tif_512x512/'
save_folder = 'Tif_256x256_StartEnd/'
save_path_mask = path_+save_folder+'masks/'
save_path_mask

'/tf/Project/LungLobe/Tif_256x256_StartEnd/masks/'

### Tiff_256x256, Tiff_512x512

In [39]:
import re
import numpy as np
from tifffile import imsave

In [27]:
#set image and mask size ==============
#@markdown Size: 128, 224, 256, 512
img_size =  256

for file in dirPath:
    path_ = os.path.join(path, file)
    print(file+' process...')
    print(path_)
    #Initiate Parameter
    lung_lobe_files = []
    Lesion_files = []
    lung_lobe_Lesion_files = []
    
    ####Create .tif function####
    dirFiles = [i for i in os.listdir(path_)]
    dirFiles.sort(key=lambda f: int(re.sub('\D', '', f)))
    
    for i in dirFiles:
    
        if 'jpg' in i: 
            file_name  = i[0:re.search('.jpg', i).span()[0]]+'.json'
            file_path = os.path.join(path_, file_name)
          
            if os.path.exists(file_path): #if already have  json
                lung_lobe, image_Lesion, lung_lobe_Lesion = json_to_matrix(file_path, img_size)
            else: #If dont have json, create matrix 0 size(img_size*img_size)
                lung_lobe = np.zeros((img_size, img_size))
                image_Lesion = np.zeros((img_size, img_size))
                lung_lobe_Lesion = np.zeros((img_size, img_size))
        
            lung_lobe_files.append(lung_lobe)
            Lesion_files.append(image_Lesion)
            lung_lobe_Lesion_files.append(lung_lobe_Lesion)
        else: continue
    
    # convert to array
    lung_lobe_files_array = np.asarray(np.array(lung_lobe_files), dtype="uint8" )
    print(lung_lobe_files_array.shape)
    # Lesion_files_array = np.asarray(np.array(Lesion_files), dtype="uint8" )
    # lung_lobe_Lesion_files_array = np.asarray(np.array(lung_lobe_Lesion_files), dtype="uint8" )
    # print(lung_lobe_Lesion_files_array.shape)
    
    # Train
    path_to_save_tiff = save_path_mask+file[:-3]+'_'+file[-2:]+'.tif'
    
    # Test1
    #  path_to_save_tiff = save_path+file+'.tif'
    # save shape (slice, 256, 256)
    imsave(path_to_save_tiff, lung_lobe_files_array) # save only lung lobe
    # imsave(path_to_save_tiff, lung_lobe_Lesion_files_array) # save lung lobe combine lesion
    print('save: '+path_to_save_tiff)
print('finish!')

Mild 08 process...
/tf/Project/LungLobe/Verification Lot 2/Mild 08


/tmp/ipykernel_700/62405243.py:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask_img = mask.astype(np.int)#boolean to 0,Convert to 1
/tmp/ipykernel_700/62405243.py:32: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecate

(95, 256, 256)


/tmp/ipykernel_700/2889547243.py:49: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  imsave(path_to_save_tiff, lung_lobe_files_array) # save only lung lobe


save: /tf/Project/LungLobe/Tif_256x256/masks/Mild_08.tif
Moderate 03 process...
/tf/Project/LungLobe/Verification Lot 2/Moderate 03
(104, 256, 256)
save: /tf/Project/LungLobe/Tif_256x256/masks/Moderate_03.tif
Moderate 06 process...
/tf/Project/LungLobe/Verification Lot 2/Moderate 06
(109, 256, 256)
save: /tf/Project/LungLobe/Tif_256x256/masks/Moderate_06.tif
Moderate 07 process...
/tf/Project/LungLobe/Verification Lot 2/Moderate 07
(113, 256, 256)
save: /tf/Project/LungLobe/Tif_256x256/masks/Moderate_07.tif
Moderate 08 process...
/tf/Project/LungLobe/Verification Lot 2/Moderate 08
(133, 256, 256)
save: /tf/Project/LungLobe/Tif_256x256/masks/Moderate_08.tif
Severe 02 process...
/tf/Project/LungLobe/Verification Lot 2/Severe 02
(111, 256, 256)
save: /tf/Project/LungLobe/Tif_256x256/masks/Severe_02.tif
Severe 03 process...
/tf/Project/LungLobe/Verification Lot 2/Severe 03
(168, 256, 256)
save: /tf/Project/LungLobe/Tif_256x256/masks/Severe_03.tif
Severe 07 process...
/tf/Project/LungLobe/V

### Tiff_256x256 เอาเฉพาะ slice ที่มีการ label

In [40]:
import re
import numpy as np
from tifffile import imsave

In [41]:
#set image and mask size ==============
#@markdown Size: 128, 224, 256, 512
img_size =  256

for file in dirPath:
    path_ = os.path.join(path, file)
    print(file+' process...')
    print(path_)
    #Initiate Parameter
    lung_lobe_files = []
    Lesion_files = []
    lung_lobe_Lesion_files = []
    
    ####Create .tif function####
    dirFiles = [i for i in os.listdir(path_)]
    dirFiles.sort(key=lambda f: int(re.sub('\D', '', f)))
    
    for i in dirFiles:
    
        if 'jpg' in i: 
            file_name  = i[0:re.search('.jpg', i).span()[0]]+'.json'
            file_path = os.path.join(path_, file_name)
          
            if os.path.exists(file_path): #if already have  json
                lung_lobe, image_Lesion, lung_lobe_Lesion = json_to_matrix(file_path, img_size)
            else: #If dont have json, skip this slice
                continue
                
            lung_lobe_files.append(lung_lobe)
            Lesion_files.append(image_Lesion)
            lung_lobe_Lesion_files.append(lung_lobe_Lesion)
        else: continue
    
    # convert to array
    lung_lobe_files_array = np.asarray(np.array(lung_lobe_files), dtype="uint8" )
    print(lung_lobe_files_array.shape)
    # Lesion_files_array = np.asarray(np.array(Lesion_files), dtype="uint8" )
    # lung_lobe_Lesion_files_array = np.asarray(np.array(lung_lobe_Lesion_files), dtype="uint8" )
    # print(lung_lobe_Lesion_files_array.shape)
    
    # Train
    path_to_save_tiff = save_path_mask+file[:-3]+'_'+file[-2:]+'.tif'
    
    # Test1
    #  path_to_save_tiff = save_path+file+'.tif'
    # save shape (slice, 256, 256)
    imsave(path_to_save_tiff, lung_lobe_files_array) # save only lung lobe
    # imsave(path_to_save_tiff, lung_lobe_Lesion_files_array) # save lung lobe combine lesion
    print('save: '+path_to_save_tiff)
print('finish!')

Mild 08 process...
/tf/Project/LungLobe/Verification Lot 2/Mild 08


/tmp/ipykernel_700/62405243.py:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask_img = mask.astype(np.int)#boolean to 0,Convert to 1
/tmp/ipykernel_700/62405243.py:32: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecate

(58, 256, 256)


/tmp/ipykernel_700/2793784858.py:47: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  imsave(path_to_save_tiff, lung_lobe_files_array) # save only lung lobe


save: /tf/Project/LungLobe/Tif_256x256_StartEnd/masks/Mild_08.tif
Moderate 03 process...
/tf/Project/LungLobe/Verification Lot 2/Moderate 03
(72, 256, 256)
save: /tf/Project/LungLobe/Tif_256x256_StartEnd/masks/Moderate_03.tif
Moderate 06 process...
/tf/Project/LungLobe/Verification Lot 2/Moderate 06
(81, 256, 256)
save: /tf/Project/LungLobe/Tif_256x256_StartEnd/masks/Moderate_06.tif
Moderate 07 process...
/tf/Project/LungLobe/Verification Lot 2/Moderate 07
(77, 256, 256)
save: /tf/Project/LungLobe/Tif_256x256_StartEnd/masks/Moderate_07.tif
Moderate 08 process...
/tf/Project/LungLobe/Verification Lot 2/Moderate 08
(86, 256, 256)
save: /tf/Project/LungLobe/Tif_256x256_StartEnd/masks/Moderate_08.tif
Severe 02 process...
/tf/Project/LungLobe/Verification Lot 2/Severe 02
(60, 256, 256)
save: /tf/Project/LungLobe/Tif_256x256_StartEnd/masks/Severe_02.tif
Severe 03 process...
/tf/Project/LungLobe/Verification Lot 2/Severe 03
(123, 256, 256)
save: /tf/Project/LungLobe/Tif_256x256_StartEnd/masks

# Image to Tif file for loop

In [3]:
import os 
import re
import numpy as np
from PIL import Image
import cv2
from tifffile import imsave

contrast function

In [4]:
# change contrast
def apply_brightness_contrast(input_img, brightness = 0, contrast = 0):
    
    if brightness != 0:
        if brightness > 0:
            shadow = brightness
            highlight = 255
        else:
            shadow = 0
            highlight = 255 + brightness
        alpha_b = (highlight - shadow)/255
        gamma_b = shadow
        
        buf = cv2.addWeighted(input_img, alpha_b, input_img, 0, gamma_b)
    else:
        buf = input_img.copy()
    
    if contrast != 0:
        f = 131*(contrast + 127)/(127*(131-contrast))
        alpha_c = f
        gamma_c = 127*(1-f)
        
        buf = cv2.addWeighted(buf, alpha_c, buf, 0, gamma_c)

    return buf

### Path setting

In [25]:
path_ = '/tf/Project/LungLobe/'
# dataset = 'Verification Lot 1'
dataset = 'Verification Lot 2'
# dataset = 'Unverified'
path = path_+dataset
path

'/tf/Project/LungLobe/Verification Lot 2'

In [26]:
dirPath = [i for i in os.listdir(path)]
dirPath.remove('.ipynb_checkpoints')
print(len(dirPath))
print(dirPath)

9
['Mild 08', 'Moderate 03', 'Moderate 06', 'Moderate 07', 'Moderate 08', 'Severe 02', 'Severe 03', 'Severe 07', 'Severe 08']


Save path img

In [30]:
save_folder = 'Tif_256x256/'
# save_folder = 'Tif_512x512/'
# save_folder = 'Tif_256x256_StartEnd/'
save_path_img = path_+save_folder+'images_createCLAHE/' # images, images_createCLAHE, images_equalizeHist
save_path_img

'/tf/Project/LungLobe/Tif_256x256/images_createCLAHE/'

### Image -> Tiff_256x256, Tiff_512,512

In [31]:
def img_make_Tiff (path, file_list, save_path_img,contrast_type, img_size, b, c):
    for file_ in file_list:
        path_ = os.path.join(path, file_)
        print(file_+' process...')
        print(path_)
      
      
        dirFiles = []
        img_3d = []
        for i in os.listdir(path_):
            dirFiles.append(i)
            dirFiles.sort(key=lambda f: int(re.sub('\D', '', f)))
      
        for file in dirFiles:
            if file[11:] == 'jpg':
                file_name  = file[:10]+'.jpg'
                file_path = os.path.join(path_, file_name)
                # print(file_path)
                if os.path.exists(file_path): #have file return => True, no file return => False
                    image = np.array(Image.open(file_path).convert('L'))
                    # resize to img_size
                    image = cv2.resize(image, (img_size, img_size), interpolation = cv2.INTER_AREA)
                    # image = resize(image, (img_size, img_size))
                    if contrast_type == 'equalizeHist':
                        # change contrast 1
                        img_contrast = cv2.equalizeHist(image)
                    elif contrast_type == 'createCLAHE':
                        # change contrast 2
                        clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8,8))
                        img_contrast = clahe.apply(image)

                    elif contrast_type == 'manually':
                        # change contrast 3
                        c = contrast
                        img_contrast = apply_brightness_contrast(image, b, c)
                    else:
                        img_contrast = image
                else: print('no image file:',file_name)
                  
                img_3d.append(img_contrast)
            else: continue
      
        # convert to array
        array_3d_img = np.array(img_3d)
        print(array_3d_img.shape)
      
        # save
        path_to_save_tiff = save_path_img+file_[:-3]+'_'+file_[-2:]+'.tif'
        # save shape (slice, 256, 256)
        imsave(path_to_save_tiff, array_3d_img)
        print('contrast_type:'+contrast_type)
        print('save: '+path_to_save_tiff)
    print('finish!')

In [32]:
# 'raw', 'createCLAHE', 'equalizeHist', 'manually'
img_make_Tiff(path, dirPath, save_path_img, 'createCLAHE', 256, 0,0)

Mild 08 process...
/tf/Project/LungLobe/Verification Lot 2/Mild 08
(98, 256, 256)


/tmp/ipykernel_707/1930640989.py:50: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  imsave(path_to_save_tiff, array_3d_img)


contrast_type:createCLAHE
save: /tf/Project/LungLobe/Tif_256x256/images_createCLAHE/Mild_08.tif
Moderate 03 process...
/tf/Project/LungLobe/Verification Lot 2/Moderate 03
(104, 256, 256)
contrast_type:createCLAHE
save: /tf/Project/LungLobe/Tif_256x256/images_createCLAHE/Moderate_03.tif
Moderate 06 process...
/tf/Project/LungLobe/Verification Lot 2/Moderate 06
(109, 256, 256)
contrast_type:createCLAHE
save: /tf/Project/LungLobe/Tif_256x256/images_createCLAHE/Moderate_06.tif
Moderate 07 process...
/tf/Project/LungLobe/Verification Lot 2/Moderate 07
(113, 256, 256)
contrast_type:createCLAHE
save: /tf/Project/LungLobe/Tif_256x256/images_createCLAHE/Moderate_07.tif
Moderate 08 process...
/tf/Project/LungLobe/Verification Lot 2/Moderate 08
(133, 256, 256)
contrast_type:createCLAHE
save: /tf/Project/LungLobe/Tif_256x256/images_createCLAHE/Moderate_08.tif
Severe 02 process...
/tf/Project/LungLobe/Verification Lot 2/Severe 02
(111, 256, 256)
contrast_type:createCLAHE
save: /tf/Project/LungLobe/

### Image -> Tiff_256x256 เอาเฉพาะภาพที่มีการ label

In [ ]:
#สร้างไฟล์ Tiff แบบทุก slice
print('data size > %d' %len(dirPath))

#@markdown Size: 128, 224, 256, 512
img_size = 256
# c =  0
# b = 0 

for file_ in dirPath:
    path_ = os.path.join(path, file_)
    print(file_+' process...')
    print(path_)
  
  
    dirFiles = []
    img_3d = []
    for i in os.listdir(path_):
        dirFiles.append(i)
        dirFiles.sort(key=lambda f: int(re.sub('\D', '', f)))
  
    for file in dirFiles:
        if file[11:] == 'json':
            file_name  = file[:10]+'.jpg'
            file_path = os.path.join(path_, file_name)
            # print(file_path)
            if os.path.exists(file_path): #have file return => True, no file return => False
              # print('OK')
              image = np.array(Image.open(file_path).convert('L'))
      
              # resize to img_size
              image = cv2.resize(image, (img_size, img_size), interpolation = cv2.INTER_AREA)
              # image = resize(image, (img_size, img_size))
      
              # change contrast
              # img_contrast = apply_brightness_contrast(image, b, c)
      
            else:
              print('no image file:',file_name)
              
            img_3d.append(image)
        else: continue
  
    # convert to array
    array_3d_img = np.array(img_3d)
    print(array_3d_img.shape)
  
    # save
    path_to_save_tiff = save_path_img+file_[:-3]+'_'+file_[-2:]+'.tif'
    # save shape (slice, 256, 256)
    imsave(path_to_save_tiff, array_3d_img)
    print('save: '+path_to_save_tiff)
print('finish!')